## Importando Bibliotecas Necessárias

links:
https://github.com/binance/binance-spot-api-docs/blob/master/rest-api.md

https://python-binance.readthedocs.io/en/latest/constants.html

In [1]:
import pandas as pd
import numpy as np
from binance.client import Client
import os

In [2]:
api_key = os.getenv(key='BINANCE_API_KEY')
api_secret = os.getenv(key='BINANCE_SECRET_KEY')

client = Client(api_key, api_secret)

## Definindo função para extração dos dados

In [3]:
def BinanceDataFrame(symbol):
    # Consultando a API da Binance
    ar = np.array(
    client.get_historical_klines(
        symbol=symbol,
        interval=Client.KLINE_INTERVAL_1DAY,
        start_str='1 Jan, 2020'
        )
    )
    
    # Transformando dados em um DataFrame
    df = pd.DataFrame(
    ar, 
    dtype=float, 
    columns=(
        'Open_Time',
        'Open_Price',
        'High_Price',
        'Low_price',
        'Close_Price',
        'Volume',
        'Close_Time',
        'Quote_Asset_Volume',
        'Number_of_Trades',
        'Taker buy base asset volume',
        'Taker buy quote asset volume',
        'Unused field'
    ))
    
    # Convertendo as colunas para datetime

    df['Open_Time'] = pd.to_datetime(df['Open_Time'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')
    df['Close_Time'] = pd.to_datetime(df['Close_Time'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')
    
    # Selecionando colunas desejadas
    df = df[[
        'Open_Time',
        'Open_Price',
        'High_Price',
        'Low_price',
        'Close_Price',
        'Volume',
        'Close_Time',
        'Number_of_Trades'
    ]]

    return df

## Utilizando a função para trazer os dados das cryptos

In [4]:
btc = BinanceDataFrame('BTCUSDT')

eth = BinanceDataFrame('ETHUSDT')

## Criando variáveis (Média Móvel e Bandas de Bollinger)

In [5]:
def Bollinger(df):
    # Média móvel
    df['Media_Movel'] = df['Close_Price'].shift(1).rolling(20).mean()

    # Desvio padrão
    df['Desvio_Padrao'] = df['Close_Price'].shift(1).rolling(20).std()

    # Banda superior
    df['Banda_Superior'] = df['Media_Movel'] + 2*btc['Desvio_Padrao']

    # Banda inferior
    df['Banda_Inferior'] = df['Media_Movel'] - 2*btc['Desvio_Padrao']

    return df

In [6]:
btc = Bollinger(btc)

eth = Bollinger(eth)

In [7]:
btc.head(25)

,Open_Time,Open_Price,High_Price,Low_price,Close_Price,Volume,Close_Time,Number_of_Trades,Media_Movel,Desvio_Padrao,Banda_Superior,Banda_Inferior
0,2019-12-31 21:00:00-03:00,7195.24,7255.00,7175.15,7200.85,16792.388165,2020-01-01 20:59:59.999000064-03:00,194010.0,NaN,NaN,NaN,NaN
1,2020-01-01 21:00:00-03:00,7200.77,7212.50,6924.74,6965.71,31951.483932,2020-01-02 20:59:59.999000064-03:00,302667.0,NaN,NaN,NaN,NaN
2,2020-01-02 21:00:00-03:00,6965.49,7405.00,6871.04,7344.96,68428.500451,2020-01-03 20:59:59.999000064-03:00,519854.0,NaN,NaN,NaN,NaN
3,2020-01-03 21:00:00-03:00,7345.00,7404.00,7272.21,7354.11,29987.974977,2020-01-04 20:59:59.999000064-03:00,279370.0,NaN,NaN,NaN,NaN
4,2020-01-04 21:00:00-03:00,7354.19,7495.00,7318.00,7358.75,38331.085604,2020-01-05 20:59:59.999000064-03:00,329209.0,NaN,NaN,NaN,NaN
5,2020-01-05 21:00:00-03:00,7357.64,7795.34,7346.76,7758.00,54635.695316,2020-01-06 20:59:59.999000064-03:00,449937.0,NaN,NaN,NaN,NaN
6,2020-01-06 21:00:00-03:00,7758.90,8207.68,7723.71,8145.28,91171.684661,2020-01-07 20:59:59.999000064-03:00,685315.0,NaN,NaN,NaN,NaN
7,2020-01-07 21:00:00-03:00,8145.92,8455.00,7870.00,8055.98,112622.642640,2020-01-08 20:59:59.999000064-03:00,830375.0,NaN,NaN,NaN,NaN
8,2020-01-08 21:00:00-03:00,8054.72,8055.96,7750.00,7817.76,64239.519830,2020-01-09 20:59:59.999000064-03:00,478800.0,NaN,NaN,NaN,NaN
9,2020-01-09 21:00:00-03:00,7817.74,8199.00,7672.00,8197.02,82406.777448,2020-01-10 20:59:59.999000064-03:00,606101.0,NaN,NaN,NaN,NaN


In [8]:
eth.head(25)

,Open_Time,Open_Price,High_Price,Low_price,Close_Price,Volume,Close_Time,Number_of_Trades,Media_Movel,Desvio_Padrao,Banda_Superior,Banda_Inferior
0,2019-12-31 21:00:00-03:00,129.16,133.05,128.68,130.77,1.447705e+05,2020-01-01 20:59:59.999000064-03:00,75888.0,NaN,NaN,NaN,NaN
1,2020-01-01 21:00:00-03:00,130.72,130.78,126.38,127.19,2.137571e+05,2020-01-02 20:59:59.999000064-03:00,96193.0,NaN,NaN,NaN,NaN
2,2020-01-02 21:00:00-03:00,127.19,135.14,125.88,134.35,4.130552e+05,2020-01-03 20:59:59.999000064-03:00,162310.0,NaN,NaN,NaN,NaN
3,2020-01-03 21:00:00-03:00,134.37,135.85,132.50,134.20,1.842762e+05,2020-01-04 20:59:59.999000064-03:00,95488.0,NaN,NaN,NaN,NaN
4,2020-01-04 21:00:00-03:00,134.20,138.19,134.19,135.37,2.541205e+05,2020-01-05 20:59:59.999000064-03:00,115106.0,NaN,NaN,NaN,NaN
5,2020-01-05 21:00:00-03:00,135.37,144.41,134.86,144.15,4.080203e+05,2020-01-06 20:59:59.999000064-03:00,159866.0,NaN,NaN,NaN,NaN
6,2020-01-06 21:00:00-03:00,144.14,145.31,138.76,142.80,4.477622e+05,2020-01-07 20:59:59.999000064-03:00,188444.0,NaN,NaN,NaN,NaN
7,2020-01-07 21:00:00-03:00,142.80,147.77,137.03,140.72,5.704656e+05,2020-01-08 20:59:59.999000064-03:00,229912.0,NaN,NaN,NaN,NaN
8,2020-01-08 21:00:00-03:00,140.76,141.50,135.30,137.74,3.660761e+05,2020-01-09 20:59:59.999000064-03:00,152241.0,NaN,NaN,NaN,NaN
9,2020-01-09 21:00:00-03:00,137.73,145.17,135.32,144.84,4.094036e+05,2020-01-10 20:59:59.999000064-03:00,167313.0,NaN,NaN,NaN,NaN
